In [ ]:
import ccxt
import pandas as pd

# 定義交易對
symbol = 'PENGU/USDT'  # 假設 PENGU 是以 USDT 計價的
listing_time = '2023-01-01T00:00:00Z'  # 假設上幣日期
since = None

# 獲取幣種的上幣時間
def get_since(exchange):
    return exchange.parse8601(listing_time)

# 獲取歷史交易數據
def fetch_ohlcv(exchange, symbol):
    since = get_since(exchange)
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe='1h', since=since)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.drop(columns=['timestamp'], inplace=True)
    return df

# 初始化交易所
exchanges = {
    'binance': ccxt.binance(),
    #'okx': ccxt.okx(),
    # 'bybit': ccxt.bybit(),
}

# 獲取數據並保存為 CSV
for exchange_name, exchange in exchanges.items():
    try:
        df = fetch_ohlcv(exchange, symbol)
        csv_filename = f'{exchange_name}_pengu_ohlcv_data.csv'
        df.to_csv(csv_filename, index=False)
        print(f"Data saved to {csv_filename}")
    except Exception as e:
        print(f"Error fetching data from {exchange_name}: {e}")

/Users/andyouyang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Data saved to binance_pengu_ohlcv_data_update.csv


In [19]:
import requests
import pandas as pd

# 定義 OKX API URL
base_url = 'https://www.okx.com/api/v5'

# 定義交易對和幣種上幣時間
symbol = 'PENGU-USDT'  # 假設 PENGU 是以 USDT 計價的
listing_time = '2023-01-01T00:00:00Z'  # 假設上幣日期

# 將上幣時間轉換為毫秒時間戳
since = int(pd.to_datetime(listing_time).timestamp() * 1000)

# 獲取歷史 OHLCV 數據
def fetch_ohlcv(symbol, since):
    url = f'{base_url}/market/candles?instId={symbol}&bar=1H&start={since}'
    response = requests.get(url)

    # 檢查請求是否成功
    if response.status_code == 200:
        data = response.json()
        if data['code'] == '0':
            return data['data']
        else:
            print(f"Error: {data['msg']}")
            return None
    else:
        print(f"HTTP Error: {response.status_code}")
        return None

# 獲取數據並保存為 CSV
ohlcv_data = fetch_ohlcv(symbol, since)
if ohlcv_data:
    # 將數據轉換為 DataFrame
    df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'count'])
    
    # 將 timestamp 轉換為可讀的日期時間格式
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms')

    # 刪除不需要的列
    df.drop(columns=['timestamp', 'close_time', 'quote_volume', 'count'], inplace=True)

    # 顯示 DataFrame
    print(df)

    # 將 DataFrame 保存為 CSV 文件
    df.to_csv('okx_pengu_ohlcv_data.csv', index=False)
    print("Data saved to okx_pengu_ohlcv_data.csv")

        open      high       low     close     volume                date
0    0.03268   0.03387  0.032541  0.033863   26164247 2024-12-19 07:00:00
1   0.032125  0.033242  0.031532  0.032664   66193157 2024-12-19 06:00:00
2   0.031676  0.033025  0.030616  0.032116  103831670 2024-12-19 05:00:00
3   0.033753  0.033762  0.031444  0.031684  103842965 2024-12-19 04:00:00
4   0.033926  0.034725  0.033059  0.033761  115798293 2024-12-19 03:00:00
5   0.033025  0.034689  0.031009  0.033959  180865225 2024-12-19 02:00:00
6    0.03421  0.034684  0.031966  0.033009  206588335 2024-12-19 01:00:00
7   0.035588  0.036919  0.033319  0.034224  162266037 2024-12-19 00:00:00
8   0.037143  0.038129  0.035267  0.035577   92247934 2024-12-18 23:00:00
9   0.036869  0.037748    0.0359  0.037159   63965319 2024-12-18 22:00:00
10  0.035731  0.037681   0.03535   0.03688   98230648 2024-12-18 21:00:00
11  0.036502   0.03757  0.035186  0.035726  205250834 2024-12-18 20:00:00
12  0.038269    0.0395  0.036312   0.0

/var/folders/ks/h0cwv1cn5t92_dn23lyd5_kw0000gn/T/ipykernel_7901/3414888606.py:38: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['date'] = pd.to_datetime(df['timestamp'], unit='ms')


In [12]:
import requests
import pandas as pd

# 定義 Bybit API URL
base_url = 'https://api.bybit.com'

# 定義交易對和上幣時間
symbol = 'PENGUUSDT'  # 假設 PENGU 是以 USDT 計價的
listing_time = '2023-01-01T00:00:00Z'  # 假設上幣日期

# 將上幣時間轉換為毫秒時間戳
since = int(pd.to_datetime(listing_time).timestamp())  # Bybit API 中的時間戳是以秒為單位

# 獲取歷史 OHLCV 數據
def fetch_ohlcv(symbol, since):
    url = f'{base_url}/v2/public/kline/list'
    params = {
        'symbol': symbol,
        'interval': '60',  # 1小時的間隔
        'from': since,
        'limit': 200,  # 每次請求的數量限制
    }
    response = requests.get(url, params=params)

    # 檢查請求是否成功
    if response.status_code == 200:
        data = response.json()
        if data['ret_code'] == 0:
            return data['result']
        else:
            print(f"Error: {data['ret_msg']}")
            return None
    else:
        print(f"HTTP Error: {response.status_code}")
        return None

# 獲取數據並保存為 CSV
ohlcv_data = fetch_ohlcv(symbol, since)
if ohlcv_data:
    # 將數據轉換為 DataFrame
    df = pd.DataFrame(ohlcv_data, columns=['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_volume', 'count'])
    
    # 將 open_time 轉換為可讀的日期時間格式
    df['date'] = pd.to_datetime(df['open_time'], unit='s')

    # 刪除不需要的列
    df.drop(columns=['open_time', 'close_time', 'quote_volume', 'count'], inplace=True)

    # 顯示 DataFrame
    print(df)

    # 將 DataFrame 保存為 CSV 文件
    df.to_csv('bybit_pengu_ohlcv_data.csv', index=False)
    print("Data saved to bybit_pengu_ohlcv_data.csv")

HTTP Error: 404


In [14]:
import ccxt
from pprint import pprint

# 创建Bybit实例，替换为您的API密钥和秘密
bybit = ccxt.bybit()

# 加载市场信息
markets = bybit.load_markets()


# 获取OHLCV数据
symbol = 'PENGU/USDT'  # 确保符号格式正确
try:
    ohlcv = bybit.fetch_ohlcv(symbol, timeframe='1h', limit=200)
    print(ohlcv)
except Exception as e:
    print(f"Error fetching OHLCV data: {e}")


[[1734444000000, 0.0034, 0.05851, 0.0034, 0.0393, 231574687.0], [1734447600000, 0.0393, 0.04217, 0.03601, 0.03661, 139399196.0], [1734451200000, 0.03661, 0.03936, 0.03133, 0.03273, 236513212.0], [1734454800000, 0.03273, 0.03299, 0.02884, 0.03105, 154654512.0], [1734458400000, 0.03105, 0.035, 0.0298, 0.03448, 147092363.0], [1734462000000, 0.03448, 0.03534, 0.03132, 0.03174, 99864040.0], [1734465600000, 0.03174, 0.03188, 0.03027, 0.03097, 61815222.0], [1734469200000, 0.03097, 0.0312, 0.02834, 0.02971, 59806291.0], [1734472800000, 0.02971, 0.031, 0.02685, 0.02775, 60102897.0], [1734476400000, 0.02775, 0.02903, 0.02711, 0.02875, 45539869.0], [1734480000000, 0.02875, 0.03066, 0.02693, 0.02722, 60085972.0], [1734483600000, 0.02722, 0.02793, 0.0265, 0.0271, 46024580.0], [1734487200000, 0.0271, 0.03188, 0.02662, 0.03106, 137488974.0], [1734490800000, 0.03106, 0.0325, 0.03027, 0.03099, 88509666.0], [1734494400000, 0.03099, 0.0319, 0.02927, 0.03049, 41907558.0], [1734498000000, 0.03049, 0.03122,

In [16]:
import ccxt
import csv
from pprint import pprint

# 创建 Bybit 实例，替换为您的 API 密钥和秘密
bybit = ccxt.bybit()

# 加载市场信息
markets = bybit.load_markets()

# 获取 OHLCV 数据
symbol = 'PENGU/USDT'  # 确保符号格式正确
try:
    ohlcv = bybit.fetch_ohlcv(symbol, timeframe='1h', limit=200)
    print(ohlcv)

    # 保存为 CSV 文件
    with open('bybit_pengu_ohlcv_data.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        # 写入表头
        writer.writerow(['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        # 写入数据
        writer.writerows(ohlcv)

    print("OHLCV 数据已成功保存为 ohlcv_data.csv")
except Exception as e:
    print(f"Error fetching OHLCV data: {e}")


[[1734444000000, 0.0034, 0.05851, 0.0034, 0.0393, 231574687.0], [1734447600000, 0.0393, 0.04217, 0.03601, 0.03661, 139399196.0], [1734451200000, 0.03661, 0.03936, 0.03133, 0.03273, 236513212.0], [1734454800000, 0.03273, 0.03299, 0.02884, 0.03105, 154654512.0], [1734458400000, 0.03105, 0.035, 0.0298, 0.03448, 147092363.0], [1734462000000, 0.03448, 0.03534, 0.03132, 0.03174, 99864040.0], [1734465600000, 0.03174, 0.03188, 0.03027, 0.03097, 61815222.0], [1734469200000, 0.03097, 0.0312, 0.02834, 0.02971, 59806291.0], [1734472800000, 0.02971, 0.031, 0.02685, 0.02775, 60102897.0], [1734476400000, 0.02775, 0.02903, 0.02711, 0.02875, 45539869.0], [1734480000000, 0.02875, 0.03066, 0.02693, 0.02722, 60085972.0], [1734483600000, 0.02722, 0.02793, 0.0265, 0.0271, 46024580.0], [1734487200000, 0.0271, 0.03188, 0.02662, 0.03106, 137488974.0], [1734490800000, 0.03106, 0.0325, 0.03027, 0.03099, 88509666.0], [1734494400000, 0.03099, 0.0319, 0.02927, 0.03049, 41907558.0], [1734498000000, 0.03049, 0.03122,